# Chile Season Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor

In [2]:
# Add the folder to the Python path

os.chdir("../../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972

c:\Users\marti\Desktop\data\hw_extra


In [3]:
def model_predict(data, regressor, len_pred, name_regressor=None, show=False):

    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data['Date'].dt.to_period('M').astype(str)

    indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
    features = data.columns.difference(["Date"]+ indices_of_interest)

    # Iterate over each index to create and evaluate models
    if show:
        fig, axs = plt.subplots(5, 1, figsize=(25,15))

    mae_errors = []
    mae_errors_training = []
    r2_scores = []
    r2_scores_training = []
    for i in range(len(indices_of_interest)):
        X = data[features]
        y = data[indices_of_interest[i]]
        dates = data["Date"]
        
        # Split into training and testing sets
        X_train, X_test, y_train, y_test = X[:-len_pred], X[-len_pred:], y[:-len_pred], y[-len_pred:]
        train_dates, test_dates = dates[:-len_pred], dates[-len_pred:]
        
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        y_pred_train = regressor.predict(X_train)
        
        # Evaluation
        mae_errors.append(mean_absolute_error(y_test, y_pred))
        mae_errors_training.append(mean_absolute_error(y_train, y_pred_train))
        r2_scores.append(r2_score(y_test, y_pred))
        r2_scores_training.append(r2_score(y_train, y_pred_train))
        
        if show:
            # Plot training values
            axs[i].plot(
                train_dates,
                y_train,
                label="Training",
                marker='o',
                color='green',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                train_dates,
                y_pred_train,
                label="Predicted Training",
                marker='x',
                color='red',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                test_dates,
                y_test,
                label="Test",
                marker='o',
                color='blue',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                test_dates,
                y_pred,
                label="Predicted Test",
                marker='x',
                color='red',
                linestyle='--',
                linewidth=1.5
            )
            axs[i].set_title(f"Prediction for {indices_of_interest[i]}")
            axs[i].legend()
    if show:
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig.suptitle(f"Model: {name_regressor}")
        plt.show()

    return [mae_errors, mae_errors_training, r2_scores, r2_scores_training]


In [4]:
data = {i: pd.read_csv(f"data/features/chile/from{FIRST_YEAR}removing/HWs_chile_features_biseason_removing_{i}.csv") for i in range(1,13)}

##  Summer DJ-JF-FM

### Linear Regression

In [5]:
linear_stats = {}

In [6]:
linear_stats[1] = model_predict(data[1], LinearRegression(), 5)

In [7]:
linear_stats[2] = model_predict(data[2], LinearRegression(), 5)

In [8]:
linear_stats[3] = model_predict(data[3], LinearRegression(), 5)

### RF

In [9]:
RF_stats = {}

In [10]:
RF_stats[1] = model_predict(data[1], RandomForestRegressor(random_state=42, n_estimators=100), 5)

In [11]:
RF_stats[2] = model_predict(data[2], RandomForestRegressor(random_state=42, n_estimators=100), 5)

In [12]:
RF_stats[3] = model_predict(data[3], RandomForestRegressor(random_state=42, n_estimators=100), 5)

### SVC

In [13]:
svc_stats = {}

In [ ]:
svc_stats[1] = model_predict(data[1], SVR(kernel='rbf') , 5)

In [ ]:
svc_stats[2] = model_predict(data[2], SVR(kernel='rbf') , 5)

In [ ]:
svc_stats[3] = model_predict(data[3], SVR(kernel='rbf') , 5)

### XGBoost

In [17]:
XGBoost_stats = {}

In [18]:
XGBoost_stats[1] = model_predict(data[1], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

In [19]:
XGBoost_stats[2] = model_predict(data[2], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

In [20]:
XGBoost_stats[3] = model_predict(data[3], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

### GaussianProcess Regressor

In [21]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
GPR_stats = {}

In [22]:
GPR_stats[1] = model_predict(data[1], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

In [23]:
GPR_stats[2] = model_predict(data[2], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

In [24]:
GPR_stats[3] = model_predict(data[3], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

### Results

In [25]:
indices_of_interest = ["Model", "Season", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_mae_df = pd.DataFrame(columns=indices_of_interest)
for i in range(1,4):
    results_mae_df.loc[len(results_mae_df.index)] = ["Linear", i] + linear_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["RF", i] + RF_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["SVC", i] + svc_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["XGBoost", i] + XGBoost_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["GPR", i] + GPR_stats[i][0]

results_mae_df.style.set_caption("MAE Model Results in Prediction").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,1,0.21,0.85,0.70,0.23,0.53
1,RF,1,0.24,1.07,0.88,0.27,0.56
2,SVC,1,0.19,1.19,0.91,0.25,0.53
3,XGBoost,1,0.26,1.27,0.82,0.30,0.64
4,GPR,1,0.23,1.11,0.90,0.25,0.50
5,Linear,2,0.48,2.27,1.47,0.68,1.41
6,RF,2,0.39,1.86,1.10,0.61,1.23
7,SVC,2,0.37,1.96,1.16,0.56,1.36
8,XGBoost,2,0.33,1.80,1.03,0.61,1.38
9,GPR,2,0.38,1.80,1.06,0.56,1.14


In [26]:
indices_of_interest = ["Model", "Season", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_mae_df_training = pd.DataFrame(columns=indices_of_interest)
for i in range(1,4):
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["Linear", i] + linear_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["RF", i] + RF_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["SVC", i] + svc_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["XGBoost", i] + XGBoost_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["GPR", i] + GPR_stats[i][1]

results_mae_df_training.style.set_caption("MAE Model Results in Training").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,1,0.12,0.48,0.39,0.19,0.36
1,RF,1,0.06,0.23,0.18,0.09,0.18
2,SVC,1,0.11,0.40,0.32,0.17,0.31
3,XGBoost,1,0.00,0.00,0.00,0.00,0.00
4,GPR,1,0.13,0.47,0.42,0.21,0.40
5,Linear,2,0.18,0.75,0.53,0.28,0.55
6,RF,2,0.08,0.33,0.25,0.13,0.25
7,SVC,2,0.13,0.64,0.46,0.21,0.46
8,XGBoost,2,0.00,0.00,0.00,0.00,0.00
9,GPR,2,0.19,0.80,0.58,0.31,0.62


In [27]:
results_r2_df = pd.DataFrame(columns=indices_of_interest)
for i in range(1,4):
    results_r2_df.loc[len(results_r2_df.index)] = ["Linear", i] + linear_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["RF", i] + RF_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["SVC", i] + svc_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["XGBoost", i] + XGBoost_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["GPR", i] + GPR_stats[i][2]
results_r2_df.style.set_caption("R^2 Model Results").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,1,-0.52,-1.16,-1.06,-0.22,-0.78
1,RF,1,-0.86,-1.67,-1.79,-0.90,-1.17
2,SVC,1,-0.40,-1.98,-1.73,-0.60,-1.13
3,XGBoost,1,-0.89,-2.15,-1.53,-0.99,-1.54
4,GPR,1,-0.67,-1.69,-1.53,-0.52,-0.86
5,Linear,2,-5.95,-4.66,-5.45,-2.33,-2.95
6,RF,2,-3.86,-3.09,-2.90,-1.64,-2.10
7,SVC,2,-3.39,-3.54,-3.20,-1.31,-2.41
8,XGBoost,2,-2.49,-3.22,-3.02,-1.73,-2.70
9,GPR,2,-3.37,-2.64,-2.39,-1.27,-1.67


In [28]:
results_r2_df_training = pd.DataFrame(columns=indices_of_interest)
for i in range(1,4):
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["Linear", i] + linear_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["RF", i] + RF_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["SVC", i] + svc_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["XGBoost", i] + XGBoost_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["GPR", i] + GPR_stats[i][3]
results_r2_df_training.style.set_caption("R^2 Model Results Training").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,1,0.47,0.46,0.45,0.42,0.41
1,RF,1,0.88,0.87,0.87,0.87,0.87
2,SVC,1,0.60,0.41,0.44,0.45,0.37
3,XGBoost,1,1.00,1.00,1.00,1.00,1.00
4,GPR,1,0.37,0.40,0.32,0.31,0.31
5,Linear,2,0.37,0.39,0.42,0.37,0.37
6,RF,2,0.88,0.88,0.87,0.87,0.87
7,SVC,2,0.67,0.37,0.44,0.55,0.31
8,XGBoost,2,1.00,1.00,1.00,1.00,1.00
9,GPR,2,0.26,0.27,0.26,0.22,0.22


##  Winter JJ-JA-AS

### Linear Regression

In [29]:
linear_stats[7] = model_predict(data[7], LinearRegression(), 5)

In [30]:
linear_stats[8] = model_predict(data[8], LinearRegression(), 5)

In [31]:
linear_stats[9] = model_predict(data[9], LinearRegression(), 5)

### RF

In [32]:
RF_stats[7] = model_predict(data[7], RandomForestRegressor(random_state=42, n_estimators=100), 5)

In [33]:
RF_stats[8] = model_predict(data[8], RandomForestRegressor(random_state=42, n_estimators=100), 5)

In [34]:
RF_stats[9] = model_predict(data[9], RandomForestRegressor(random_state=42, n_estimators=100), 5)

### SVC

In [35]:
svc_stats[7] = model_predict(data[7], SVR(kernel='rbf') , 5)

In [36]:
svc_stats[8] = model_predict(data[8], SVR(kernel='rbf') , 5)

In [37]:
svc_stats[9] = model_predict(data[9], SVR(kernel='rbf') , 5)

### XGBoost

In [38]:
XGBoost_stats[7] = model_predict(data[7], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

In [39]:
XGBoost_stats[8] = model_predict(data[8], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

In [40]:
XGBoost_stats[9] = model_predict(data[9], XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), 5)

### GPR

In [41]:
GPR_stats[7] = model_predict(data[7], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

In [42]:
GPR_stats[8] = model_predict(data[8], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

In [43]:
GPR_stats[9] = model_predict(data[9], GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10), 5)

### Results

In [44]:
indices_of_interest = ["Model", "Season", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_mae_df = pd.DataFrame(columns=indices_of_interest)
for i in range(7,10):
    results_mae_df.loc[len(results_mae_df.index)] = ["Linear", i] + linear_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["RF", i] + RF_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["SVC", i] + svc_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["XGBoost", i] + XGBoost_stats[i][0]
    results_mae_df.loc[len(results_mae_df.index)] = ["GPR", i] + GPR_stats[i][0]

results_mae_df.style.set_caption("MAE Model Results in Prediction").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,7,0.06,0.30,0.21,0.14,0.18
1,RF,7,0.06,0.26,0.19,0.11,0.17
2,SVC,7,0.05,0.27,0.18,0.18,0.21
3,XGBoost,7,0.11,0.41,0.31,0.19,0.32
4,GPR,7,0.06,0.23,0.17,0.17,0.27
5,Linear,8,0.18,0.64,0.45,0.44,0.67
6,RF,8,0.17,0.63,0.42,0.48,0.79
7,SVC,8,0.19,0.72,0.52,0.48,0.80
8,XGBoost,8,0.21,0.65,0.51,0.59,0.99
9,GPR,8,0.16,0.66,0.43,0.39,0.81


In [45]:
indices_of_interest = ["Model", "Season", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_mae_df_training = pd.DataFrame(columns=indices_of_interest)
for i in range(7,10):
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["Linear", i] + linear_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["RF", i] + RF_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["SVC", i] + svc_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["XGBoost", i] + XGBoost_stats[i][1]
    results_mae_df_training.loc[len(results_mae_df_training.index)] = ["GPR", i] + GPR_stats[i][1]

results_mae_df_training.style.set_caption("MAE Model Results in Training").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,7,0.08,0.30,0.26,0.14,0.26
1,RF,7,0.03,0.13,0.11,0.06,0.11
2,SVC,7,0.08,0.22,0.19,0.11,0.19
3,XGBoost,7,0.00,0.00,0.00,0.00,0.00
4,GPR,7,0.08,0.30,0.26,0.15,0.26
5,Linear,8,0.14,0.58,0.43,0.28,0.51
6,RF,8,0.06,0.23,0.17,0.10,0.18
7,SVC,8,0.12,0.47,0.36,0.22,0.42
8,XGBoost,8,0.00,0.00,0.00,0.00,0.00
9,GPR,8,0.15,0.55,0.45,0.28,0.40


In [46]:
results_r2_df = pd.DataFrame(columns=indices_of_interest)
for i in range(7,10):
    results_r2_df.loc[len(results_r2_df.index)] = ["Linear", i] + linear_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["RF", i] + RF_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["SVC", i] + svc_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["XGBoost", i] + XGBoost_stats[i][2]
    results_r2_df.loc[len(results_r2_df.index)] = ["GPR", i] + GPR_stats[i][2]
results_r2_df.style.set_caption("R^2 Model Results").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,7,-0.08,-0.27,-0.31,0.03,0.21
1,RF,7,-0.13,-0.14,-0.01,0.47,0.50
2,SVC,7,0.04,-0.50,-0.38,-0.14,0.23
3,XGBoost,7,-1.12,-0.73,-0.96,-0.71,-1.00
4,GPR,7,0.08,0.02,0.15,-0.19,-0.14
5,Linear,8,-1.31,-0.78,-1.56,-1.92,-1.32
6,RF,8,-1.23,-0.73,-1.13,-2.63,-2.45
7,SVC,8,-1.60,-1.31,-2.12,-2.85,-2.72
8,XGBoost,8,-2.00,-0.77,-1.52,-4.41,-4.53
9,GPR,8,-0.95,-1.14,-1.32,-1.18,-3.19


In [47]:
results_r2_df_training = pd.DataFrame(columns=indices_of_interest)
for i in range(7,10):
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["Linear", i] + linear_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["RF", i] + RF_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["SVC", i] + svc_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["XGBoost", i] + XGBoost_stats[i][3]
    results_r2_df_training.loc[len(results_r2_df_training.index)] = ["GPR", i] + GPR_stats[i][3]
results_r2_df_training.style.set_caption("R^2 Model Results in Training").format(precision=2)

,Model,Season,HWN,HWF,HWD,HWM,HWA
0,Linear,7,0.49,0.44,0.44,0.43,0.42
1,RF,7,0.89,0.88,0.89,0.88,0.88
2,SVC,7,0.64,0.59,0.62,0.59,0.53
3,XGBoost,7,1.00,1.00,1.00,1.00,1.00
4,GPR,7,0.44,0.40,0.38,0.37,0.37
5,Linear,8,0.35,0.31,0.30,0.31,0.32
6,RF,8,0.89,0.90,0.90,0.92,0.92
7,SVC,8,0.47,0.27,0.31,0.52,0.48
8,XGBoost,8,1.00,1.00,1.00,1.00,1.00
9,GPR,8,0.25,0.30,0.23,0.25,0.60
